### 参考

[1] https://zhuanlan.zhihu.com/p/197476119

[2] https://www.bilibili.com/video/BV1Mh411M7nV/?spm_id_from=333.337.search-card.all.click&vd_source=4e27374e03bfc58f9262019338fe1fa0

[3]https://www.cnblogs.com/lsm-boke/p/12256686.html

[4]https://www.knowledgedict.com/tutorial/sklearn-dataset.html

[5]决策树2 基尼系数、基尼系数增益 - empirexu的文章 - 知乎
https://zhuanlan.zhihu.com/p/456351465

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from graphviz import Digraph
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold


In [7]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None,pro_value=None):
       
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value 
        self.pro_value=pro_value

## 预剪枝决策树

In [8]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2,max_depth=2,percent=10):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.percent=percent
        self.actual_tree_depth = 0  # 实际树深度

    def build_tree(self, dataset, curr_depth=0, X_val=None, y_val=None):
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        # 统计类别分布
        class_counts = np.unique(Y, return_counts=True)
        class_distribution = dict(zip(class_counts[0], class_counts[1]))
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # 检查类别分布条件
            if len(class_distribution) > 1:  # 至少有两个类别
                major_class_count = max(class_distribution.values())
                minor_class_count = min(class_distribution.values())
                if minor_class_count > 0 and major_class_count / minor_class_count > self.percent:
                    print(f"预剪枝：深度 {curr_depth}，类别分布 {class_distribution} -> 停止分裂")
                    leaf_value = self.calculate_leaf_value(Y)
                    return Node(value=leaf_value)
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split and "info_gain" in best_split and best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1, X_val, y_val)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1, X_val, y_val)
                print(f"当前深度 {curr_depth}, 特征 {best_split['feature_index']}, 阈值 {best_split['threshold']:.2f}")
                print(f"  样本数: {len(best_split['dataset'])}, 类别分布: {best_split['class_distribution']} ")
                print(f"  样本数最多的类: {best_split['value']}")
                print(f"  左子树样本数: {len(best_split['dataset_left'])}, 类别分布: {best_split['left_class_distribution']} ")
                print(f"  右子树样本数: {len(best_split['dataset_right'])}, 类别分布: {best_split['right_class_distribution'] }")
                current_node = Node(best_split["feature_index"], best_split["threshold"],
                                    left_subtree, right_subtree, best_split["info_gain"])
                current_node.left_class_distribution = best_split["left_class_distribution"]
                current_node.right_class_distribution = best_split["right_class_distribution"]
                current_node.class_distribution = best_split["class_distribution"]
                current_node.pro_value = best_split["value"]
                return current_node
        # 若无法继续分裂，返回叶节点
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        for feature_index in range(num_features):
            feature_values = np.sort(dataset[:, feature_index])[::-1]  
            possible_thresholds = np.unique(feature_values)  
            # 计算每相邻两个值的平均值，并将其添加到 possible_thresholds 中
            average_thresholds = (feature_values[:-1] + feature_values[1:]) / 2  # 相邻元素的平均值
            possible_thresholds = np.unique(np.concatenate((possible_thresholds, average_thresholds)))
            # 合并并去重
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "else")
                    if curr_info_gain>max_info_gain:
                        best_split["dataset"]=dataset
                        classes=np.unique(best_split["dataset"][:, -1], return_counts=True)
                        best_split["class_distribution"] = dict(zip(classes[0], classes[1]))
                        majority_class = max(best_split["class_distribution"], key=best_split["class_distribution"].get)
                        best_split["value"] = majority_class
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        left_classes = np.unique(best_split["dataset_left"][:, -1], return_counts=True)
                        right_classes = np.unique(best_split["dataset_right"][:, -1], return_counts=True)
                        best_split["left_class_distribution"] = dict(zip(left_classes[0], left_classes[1]))
                        best_split["right_class_distribution"] = dict(zip(right_classes[0], right_classes[1]))
                        max_info_gain = curr_info_gain
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        # 左边是某个特征值小于阈值的样本，右边是某个特征值大于阈值的样本
        dataset_left = np.array([row for row in dataset if row[feature_index]<threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>=threshold])
        return dataset_left, dataset_right
    
    # 计算信息增益
    def information_gain(self, parent, l_child, r_child, mode="gini"):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=='gini':
            info_gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) 
                                                   + weight_r*self.gini_index(r_child))
        else:
            info_gain = self.entropy(parent) - (weight_l*self.entropy(l_child) 
                                                + weight_r*self.entropy(r_child))
        return info_gain
    # 计算基尼系数
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / (len(y))
            gini += p_cls**2
        return 1 - gini
    # 计算熵
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def calculate_tree_depth(self, node):
        """
        计算树的实际深度（递归遍历树）。
        """
        if node is None or node.value is not None:  # 叶节点深度为 1
            return 0
        left_depth = self.calculate_tree_depth(node.left)
        right_depth = self.calculate_tree_depth(node.right)
        return 1 + max(left_depth, right_depth)
    
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
   
    def fit(self, X, Y, X_val=None, y_val=None):
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)  # 确保 Y 是二维数组
        # 将数据拼接为一个整体
        dataset = np.concatenate((X, Y), axis=1)
        # 构建决策树
        self.root = self.build_tree(dataset, X_val=X_val, y_val=y_val)
        self.actual_tree_depth = self.calculate_tree_depth(self.root)-1
        print(f"实际树深度: {self.actual_tree_depth}")
    def predict(self, X):
        if self.root is None:
            print("空树")
            return [None] * len(X)  
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        if tree is None:
            raise ValueError("空树")
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def visualize_tree(self, tree=None, dot=None, node_id=0):
        if dot is None:
            dot = Digraph()
            dot.node(name=str(node_id), label="Root")  # 设置根节点的名称
        if tree is None:
            tree = self.root
        if tree.value is not None:
            dot.node(name=str(node_id), label="Class: " + str(tree.value))
        else:
            left_distribution = tree.left_class_distribution if tree.left_class_distribution else {}
            right_distribution = tree.right_class_distribution if tree.right_class_distribution else {}
            left_dist_str = ", ".join([f"{int(key)}: {value}" for key, value in left_distribution.items()])
            right_dist_str = ", ".join([f"{int(key)}: {value}" for key, value in right_distribution.items()])
            label = f"X_{tree.feature_index} <= {tree.threshold:.2f}\\ninfo gain: {tree.info_gain:.2f}"
            label += f"\\nLeft: {left_dist_str}\\nRight: {right_dist_str}"
            dot.node(name=str(node_id), label=label)
            left_child_id = node_id * 2 + 1
            dot.edge(str(node_id), str(left_child_id), label="True")  # 左分支
            self.visualize_tree(tree.left, dot, left_child_id)
            right_child_id = node_id * 2 + 2
            dot.edge(str(node_id), str(right_child_id), label="False")  # 右分支
            self.visualize_tree(tree.right, dot, right_child_id)
        return dot
    
    def prune(self, node, X_val, y_val,curr_depth=0,tree_depth=None):
        if node.left is None and node.right is None: return
        if node.left:
            self.prune(node.left, X_val, y_val,curr_depth+1,tree_depth)
        if node.right:
            self.prune(node.right, X_val, y_val,curr_depth+1,tree_depth)
        if curr_depth == tree_depth :
            be_predictions = self.predict(X_val)
            left = node.left
            right = node.right
            node.left = None
            node.right = None
            node.value = node.pro_value
            predictions = self.predict(X_val)
            y_val = np.array(y_val).ravel()  # 确保 y_val 是一维数组
            predictions = np.array(predictions).ravel()  # 确保预测结果也是一维数组
            label_encoder = LabelEncoder()
            y_val_encoded = label_encoder.fit_transform(y_val)
            be_predictions_encoded = label_encoder.transform(be_predictions)
            predictions_encoded = label_encoder.transform(predictions)
            acc_pruned = accuracy_score(y_val_encoded, predictions_encoded)
            acc_unpruned = accuracy_score(y_val_encoded, be_predictions_encoded)  # 保留未剪枝节点的准确性
            print(f"剪枝特征: {node.feature_index}, 剪枝前: {acc_unpruned}, 剪枝后: {acc_pruned}")
            if acc_pruned <= acc_unpruned:
                node.left = left
                node.right = right
                node.value = None
            else:
                print(f"特征 {node.feature_index} 剪枝后性能提高。")
        
    def fit_with_pruning(self, X, Y, X_val, y_val):
        self.fit(X, Y)
        for i in range(0,self.actual_tree_depth):
             self.prune(self.root, X_val, y_val, curr_depth=0,
                         tree_depth=self.actual_tree_depth-i)

        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth)
        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth-1)
        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth-2)
        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth-3)
        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth-4)
        # self.prune(self.root, X_val, y_val, curr_depth=0, tree_depth=self.actual_tree_depth-5)
        # 然后进行剪枝
        # for i in range(int(self.actual_tree_depth),1):
        #     self.prune(self.root, X_val, y_val, curr_depth=0,tree_depth=i)
        #     print(f"i: {i}")
            
    


    
    


## 从文件导入数据

In [9]:
# Step 1: 从 CSV 文件读取数据
file1_path = './dataset_1/X_train.csv'  # 替换为特征数据的 CSV 文件路径
file2_path = './dataset_1/y_train.csv'  # 替换为标签数据的 CSV 文件路径
file3_path = './dataset_1/X_test.csv'  # 替换为特征数据的 CSV 文件路径
file4_path = './dataset_1/y_test.csv'  # 替换为标签数据的 CSV 文件路径
# 使用 pandas 读取 CSV 文件
X_train = pd.read_csv(file1_path)  # 假设文件包含标题行
y_train = pd.read_csv(file2_path)  # 假设文件包含标题行
X_test = pd.read_csv(file3_path)  # 假设文件包含标题行
y_test = pd.read_csv(file4_path)  # 假设文件包含标题行
print("特征数据 X:")
print(X_train.head())
print("\n标签数据 y:")
print(y_train.head())
X_train=X_train.iloc[:, :4].values
print(X_train[:5])
X_test=X_test.iloc[:, :4].values

y_train = y_train.iloc[:, 0].values.reshape(-1,1)
print(y_train[:5])
y_test = y_test.iloc[:, 0].values.reshape(-1,1)


特征数据 X:
   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        15.66         23.20          110.20      773.5          0.11090   
1        15.46         23.95          103.80      731.3          0.11830   
2        11.47         16.03           73.02      402.7          0.09076   
3        19.55         28.77          133.60     1207.0          0.09260   
4        12.03         17.93           76.09      446.0          0.07683   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.31140        0.317600             0.137700         0.2495   
1           0.18700        0.203000             0.085200         0.1807   
2           0.05886        0.025870             0.023220         0.1634   
3           0.20630        0.178400             0.114400         0.1893   
4           0.03892        0.001546             0.005592         0.1382   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0   

## 数据导入

In [10]:
# Step1:导入数据
from sklearn import datasets
cancer = datasets.load_breast_cancer()
X_c = cancer.data # 特征数据
y_c = cancer.target # 标签数据
y_c_re=y_c[:,np.newaxis]

# Step2:划分数据集,后缀为0的是由sklearn库里面得到的数据集
from sklearn.model_selection import train_test_split
X_train0, X_test0, y_train0, y_test0 = train_test_split(X_c, y_c, test_size=0.2, random_state=33)

# Step3:得到决策树
classifier_c = DecisionTreeClassifier(min_samples_split=5, max_depth=5,percent=50)
# classifier_c.fit(X_train,y_train)
# classifier_c.fit(X_train0,y_train0)

# Step4:决策树可视化
# dot_c = classifier_c.visualize_tree()
# dot_c.render("decision_tree_visualize_c", format="png")  # 保存为 PNG 文件

# Step5:准确率
# y_pred = classifier_c.predict(X_test) 
# accuracy_score(y_test, y_pred)
# y_pred0 = classifier_c.predict(X_test0) 
# accuracy_score(y_test0, y_pred0)

# 后剪枝
classifier_c.fit_with_pruning(X_train, y_train,X_test, y_test)
dot_cp = classifier_c.visualize_tree()
dot_cp.render("decision_tree_visualize_cp", format="png")


当前深度 5, 特征 1, 阈值 15.58
  样本数: 106, 类别分布: {0.0: 9, 1.0: 97} 
  样本数最多的类: 1.0
  左子树样本数: 1, 类别分布: {0.0: 1} 
  右子树样本数: 105, 类别分布: {0.0: 8, 1.0: 97}
当前深度 4, 特征 0, 阈值 11.19
  样本数: 107, 类别分布: {0.0: 10, 1.0: 97} 
  样本数最多的类: 1.0
  左子树样本数: 1, 类别分布: {0.0: 1} 
  右子树样本数: 106, 类别分布: {0.0: 9, 1.0: 97}
当前深度 3, 特征 2, 阈值 73.29
  样本数: 147, 类别分布: {0.0: 10, 1.0: 137} 
  样本数最多的类: 1.0
  左子树样本数: 40, 类别分布: {1.0: 40} 
  右子树样本数: 107, 类别分布: {0.0: 10, 1.0: 97}
当前深度 2, 特征 1, 阈值 15.54
  样本数: 219, 类别分布: {0.0: 10, 1.0: 209} 
  样本数最多的类: 1.0
  左子树样本数: 72, 类别分布: {1.0: 72} 
  右子树样本数: 147, 类别分布: {0.0: 10, 1.0: 137}
当前深度 5, 特征 2, 阈值 77.40
  样本数: 16, 类别分布: {0.0: 1, 1.0: 15} 
  样本数最多的类: 1.0
  左子树样本数: 13, 类别分布: {1.0: 13} 
  右子树样本数: 3, 类别分布: {0.0: 1, 1.0: 2}
当前深度 4, 特征 1, 阈值 21.30
  样本数: 17, 类别分布: {0.0: 2, 1.0: 15} 
  样本数最多的类: 1.0
  左子树样本数: 16, 类别分布: {0.0: 1, 1.0: 15} 
  右子树样本数: 1, 类别分布: {0.0: 1}
当前深度 3, 特征 1, 阈值 21.38
  样本数: 47, 类别分布: {0.0: 2, 1.0: 45} 
  样本数最多的类: 1.0
  左子树样本数: 17, 类别分布: {0.0: 2, 1.0: 15} 
  右子树样本数: 30, 类别分布: {

'decision_tree_visualize_cp.png'